## CNN TRAINING SAMPLES PREPERATION

In [2]:
##code for preparing the training data.
""""adapted from the original code developed in Bergado, J. R. A., Persello, C., & Gevaert, C. (2016). 
    A deep learning approach to the classification of sub-decimeter resolution aerial images.
    In IEEE International Geoscience and Remote Sensing Symposium (pp. 1516–1519). Beijing."""


'"adapted from the coded developed in Bergado, J. R. A., Persello, C., & Gevaert, C. (2016). \n    A deep learning approach to the classification of sub-decimeter resolution aerial images.\n    In IEEE International Geoscience and Remote Sensing Symposium (pp. 1516\xe2\x80\x931519). Beijing.'

In [3]:
##importing libraries and dependencies
from __future__ import division, print_function
import math
import os
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
from PIL import Image
from osgeo import gdal

In [4]:
## defining the functions

In [5]:
#functions for loading the images and converting them to arrays

def img_to_array(*images):
    """Convert an image or list of images to numpy arrays.

    Keyword arguments:
    *images -- list containing the images to be converted
    """
    imgarrays = []
    i = 0
    for img in images:
        arr = gtiff_to_array(img)
        imgarrays.append(arr)
    return imgarrays

In [6]:
# convert the geotiff to a numpy array

def gtiff_to_array(imgfname):
    """Transform a geotiff to numpy array.

    Keyword arguments:
    imgfnames -- filename of image to convert
    """
    ds = gdal.Open(imgfname)
    for band in range(ds.RasterCount):
        band += 1
        if band == 1:
            arr = np.array(ds.GetRasterBand(band).ReadAsArray())
            arr = np.expand_dims(arr, axis=2)
        else:
            concat = np.array(ds.GetRasterBand(band).ReadAsArray())
            concat = np.expand_dims(concat, axis=2)
            arr = np.concatenate((arr,
                                  concat),
                                 axis=2)
    return arr

In [7]:
# normalize the top array

def norm_data(data):
    data = data.astype(float)
    l_max = np.amax(data)
    l_min = np.amin(data)
    #l_max = 2424
    #l_min = 7
    data_norm = (data - l_min)/(l_max - l_min)
    return data_norm

In [8]:
# removing the null values and assigning them to 0"

def nulltozero(arr):
    arrcopy = np.copy(arr)
    low_values_flags = arrcopy < 0 #0.7,0
    arrcopy[low_values_flags]=0
    return arrcopy
    

In [7]:
# sampling an array based on the class frequency
def sample_idx(arr):
    """Randomly sample an array stratified based on frequency.

    Keyword arguments:
    arr -- the array being sampled
    cratios -- the representative fractions of each classes
    n -- total number of samples (default 1000)
    
    i = 0
    arr_copy = np.copy(arr)
    idxarray = np.zeros(shape=(0, 2), dtype=np.int16)
    nc = np.array(np.where(arr_copy >0)).T.shape[0]
    #sampleidx = 0
    #nc = np.array(np.where(arr_copy == i+1 or arr_copy == i+2)).T.shape[0]
    #nc = np.array(np.logical_or(arr_copy == i+1, arr_copy == i+2)).T.shape[0]
    #find how to automatically select the samples
    randidx = np.random.choice(range(nc),
							   size=nc, replace=False)
    #print()
	#randidx.dtype = np.int32
    #idxarray = np.concatenate((idxarray,
	#						   np.array(np.where(arr_copy == i+1)).T
	#						   [randidx, :]),
	#						  axis=0)
    #idxarray = np.array(np.where(arr_copy == i+1)).T[randidx, :]
    randidx=randidx.astype(np.int32)
    idxarray = np.array(np.where(arr_copy > 0)).T[randidx, :]
	#sampleidx += csamples

    del arr_copy
    return idxarray
    """

In [9]:
# sampling an array
def sample_idx(arr):
    """Randomly sample an array

    
    """
    i = 0
    arr_copy = np.copy(arr)
    idxarray = np.zeros(shape=(0, 2), dtype=np.int16)
    nc = np.array(np.where(arr_copy >0)).T.shape[0]
    #find how to automatically select the samples
    randidx = np.random.choice(range(nc),
							   size=nc, replace=False)
    randidx=randidx.astype(np.int32)
    idxarray = np.array(np.where(arr_copy > 0)).T[randidx, :]

    del arr_copy
    return idxarray

In [10]:
"find a way of skipping this step"
# not call this function
#build the training and test sets

def split_samples(idxarray, proportion=0.67):
    """Split indices into two sets.

    Keyword arguments:
    idxarray -- the array of indices to be split
    proportion -- the proportion of split
    """
    nsamples = idxarray.shape[0]
    nsub1 = int(round(proportion*nsamples))
    randidx = np.random.choice(range(nsamples), size=nsub1, replace=False)
	#setting randidx data type to integer, the other option is to be a boolean
    randidx.dtype = np.int32
    idxsub1 = idxarray[randidx, :]
    randidx = list(set(range(nsamples)) - set(randidx))
    idxsub2 = idxarray[randidx, :]

    return (idxsub1, idxsub2)

In [11]:
"define the function for accumulating samples"
def accum_samples_cnn(accumarray, fset, idxarray, psize, sparse=True,
                      ignoreedge=125):
    """Accumulate samples from different fset for a cnn model.

    Keyword arguments:
    accumarray -- the array containing accumulated samples
    fset -- the tuple containg feature set arrays
    idxarray -- the array containing indices of samples
    psize - the size of the context patch
    sparse -- a flag used for the type of sampling. Set to False to
    sample whole image (by batch) and hence using ntrain and ntest.
    Returns the indices when sparse is set to False.
    """
    n = idxarray.shape[0]
    include = int(math.floor(psize/2))
    exclude = include

    # limit sampling using the max patch size for ignoring pixels
    if ignoreedge:
        exclude = int(math.floor(ignoreedge/2))
    # gather patches without zero-padding
    nonedgesamples = []
    f0 = fset[0]
    print("f0 shape",f0.shape)
    nonedgeidx = np.asarray([rc
                             for rc in idxarray if
                             ((rc[0] > exclude - 1) and
                              (rc[1] > exclude - 1) and
                              (rc[0] < f0.shape[0] - exclude) and
                              (rc[1] < f0.shape[1] - exclude))
                             ])

    for f in fset:
        nonedgesamples.append(np.asarray([f[rc[0]-include:rc[0]+include+1,
                                            rc[1]-include:rc[1]+include+1, ]
                                          for rc in nonedgeidx]))
    #print("nonedgesamples",nonedgesamples)
    # gather patches with zero-padding
    edgeidx = np.asarray([rc
                          for rc in idxarray if
                          ((rc[0] <= exclude - 1) or
                           (rc[1] <= exclude - 1) or
                           (rc[0] >= f0.shape[0] - exclude) or
                           (rc[1] >= f0.shape[1] - exclude))
                          ])
    #jr case,ncols evaluates to 5
	#my case, f0 has 4 bands ,therefore evaluates to 6. so i will add 1 instead of 2
	#ncols = f0.shape[2] + 2
    #ncols = f0.shape[2] + 1 ncols is the numbero bands plus 1
    ncols=4
    print("ncols",ncols)
    if edgeidx.shape[0] != 0:
        edgesamples = build_edge_samples(edgeidx, fset, include)
        edgefset = np.concatenate(edgesamples, axis=3)
    else:
        edgefset = np.zeros(shape=(0, psize, psize, ncols))

    # gather all features
    # check for small batches consisting of all edge pixels
	print("noneedgesamples shape",len(nonedgesamples))
    print("nonedgesamples[0].shape[0]",nonedgesamples[0].shape[0])
    #print("noneedgesamples ",nonedgesamples)
    """if nonedgesamples[0].shape[0] != 0:
        nonedgefset = np.concatenate((nonedgesamples[0],
                                      nonedgesamples[1],
                                      np.expand_dims(nonedgesamples[2],
                                                     axis=3)),
                                     axis=3)"""
    #print("nonedgesamples",nonedgesamples[0])
    if nonedgesamples[0].shape[0] != 0:
        nonedgefset = np.concatenate((nonedgesamples[0],
                                      np.expand_dims(nonedgesamples[1], 
                                      
                                                     axis=3)),
                                     axis=3)
	"""if nonedgesamples[0].shape[0] != 0:
        print("this is so far true")
        nonedgefset = np.concatenate((nonedgesamples[0],
                                      np.expand_dims(nonedgesamples[1],axis=3)),
                                     axis=3)"""
    else:
        print("else statement executed")
        nonedgefset = np.zeros(shape=(0, psize, psize, ncols))
        #print("nonedgefset shape",nonedgefset.shape)
    #print nonedgefset
    #print("nonedgefset",nonedgefset)
	#print nonedgefset shape
    print("nonedgefset shape",nonedgefset.shape)
	# gather all samples
    if ignoreedge:
        accumarray = np.concatenate((accumarray, nonedgefset), axis=0)
        idxarray = nonedgeidx
		
    else:
        accumarray = np.concatenate((accumarray, nonedgefset, edgefset),
                                    axis=0)
        if nonedgeidx.shape[0] == 0:
            nonedgeidx = nonedgeidx.reshape(0, 2)
        if edgeidx.shape[0] == 0:
            edgeidx = edgeidx.reshape(0, 2)
        idxarray = np.concatenate((nonedgeidx, edgeidx), axis=0)

    return (accumarray, idxarray)


In [12]:
def build_edge_samples(idxarray, fset, pad):
    """Pad zeros to patches centered in edge and near-edge pixels.

    Keyword arguments:
    idxarray -- the index array of edge and near-edge pixels
    fset -- the tuple containg feature set arrays
    pad -- the number of zeros to pad on each side of the image
    """
    paddedfset = []
    for f in fset:
        if f.ndim != 2:
            padded = np.zeros((f.shape[0] + 2*pad,
                               f.shape[1] + 2*pad, f.shape[2]),
                              dtype=f.dtype)
            padded[pad:f.shape[0]+pad,
                   pad:f.shape[1]+pad, ] = f
        else:
            padded = np.zeros((f.shape[0] + 2*pad,
                               f.shape[1] + 2*pad, 1),
                              dtype=f.dtype)
            padded[pad:f.shape[0]+pad,
                   pad:f.shape[1]+pad, ] = f.reshape((f.shape[0],
                                                      f.shape[1],
                                                      1))
        paddedfset.append(padded)
    edgesamples = []
    for f in paddedfset:
        edgesamples.append(np.asarray([f[rc[0]:rc[0]+2*pad+1,
                                         rc[1]:rc[1]+2*pad+1, ]
                                       for rc in idxarray
                                       ]))

    return edgesamples

In [13]:
def save_samplesets(alltrainset, fname,idfname,
                    nc=2, tp=0.5, labels="central"):
    """Save the splitted set of samples in an hdf5 file.

    alltrainset -- the array containing all the training samples,
    this will be split into training and validation sets
    alltestset -- the array containing all test samples
    fname -- the filename (and directory) where to write the hdf5
    nc -- the number of classes
    tp -- the proportional split of training samples
    from alltrainset
    """
    nf = alltrainset.shape[1] - 1

    # separate X's (features) and y's (labels)
    (X_train, y_train) = (alltrainset[:, 0:nf, ],
                          alltrainset[:, nf, ])
    #(X_test, y_test) = (alltestset[:, 0:nf, ],
    #                    alltestset[:, nf, ])
    # take only the label of the central pixel
    #span = int(math.floor(y_train.shape[2]/2))
    #if labels=="central":
    #    y_train = y_train[:, span, span]
    #    y_test = y_test[:, span, span]

    X_train = X_train.astype("float32")
    #X_test = X_test.astype("float32")
    # convert class vectors to binary class matrices
    #if labels=="central":
    #    Y_train = np_utils.to_categorical(y_train, nc)
    #    Y_test = np_utils.to_categorical(y_test, nc)
    #elif labels=="full":
    #    Y_train = to_categorical_4d(y_train, nc)
    #    Y_test = to_categorical_4d(y_test, nc)
                

    # split trainset into training and validation
    origidx = np.arange(X_train.shape[0]).reshape(X_train.shape[0], 1)
    trainidx, validx = split_samples(origidx,
                                     tp)
    trainidx = trainidx.reshape(trainidx.shape[0])
    validx = validx.reshape(validx.shape[0])
	
	#save the indices of the training sets
    
    with h5py.File(idfname,mode = "w") as f:
	    f["trainidx"]=trainidx
	    f["validx"] =validx
    
	
	
    X_val = X_train[validx, :]
    #Y_val = Y_train[validx, :]
    y_val = y_train[validx]
    X_train = X_train[trainidx, :]
    #Y_train = Y_train[trainidx, :]
    y_train = y_train[trainidx]

    y_train = np.expand_dims(y_train, axis=1)
    y_val = np.expand_dims(y_val, axis=1)
    #y_test = np.expand_dims(y_test, axis=1)

    with h5py.File(fname, mode="w") as f:
        f["X_train"] = X_train
        f["X_val"] = X_val
        #f["X_test"] = X_test
        #f["Y_train"] = Y_train
        #f["Y_val"] = Y_val
        #f["Y_test"] = Y_test
        #f["y_test"] = y_test
        f["y_train"] = y_train
        f["y_val"] = y_val

In [14]:
##paths

"paths to the new dataset"
rootfolder = "H:/BUKAVU-2018/PRELIM_CNN/"
raster_data_path = "H:/tiles/grayscale/tile2cnn/"
#tile2=raster_data_path+"tile2_v2_gray.tif"
#tile2 = "H:/tiles/grayscale/tile2_v4_gray_n.tif"
tile2 = "H:/goma_train_seg/TRAINING_TILE/TILE2_clip.tif"
#tile2 = "H:/goma_train_seg/TRAINING_TILE/TILE2_clip_geobia2.tif"

#tile2_dsm = raster_data_path+"tile2_ndsm_50_clip.tif"

gts_paths = "H:/BUKAVU-2018/PRELIM_CNN/GTS_IMAGES/"
training_pts=gts_paths+"training_pts_ndsm50.tiff"
training_vector=gts_paths+"training_vector_ndsm50.tiff"

#classified_raster = "H:/goma_train_seg/classified_raster/TILE2_clip_geobia2.tif"
classified_raster = "H:/goma_train_seg/TRAINING_TILE/TILE_clip_geobia2.tif"

tutorialdir = "H:/tutorial_fcn/"
smpldir= tutorialdir  + "samples/samples_wfcn_v1.hdf5"
inddir = tutorialdir  + "samples/samples_indir_wfcn_v1.hdf5"


In [15]:
# tile2_cr is the classified raster
#TO DO -find a way to automatically mask the computation region
#http://karthur.org/2015/clipping-rasters-in-python.html
tile2_top,tile2_points,tile2_poly,tile2_cr = img_to_array(tile2,training_pts,training_vector,classified_raster)


In [27]:
#tile2_top.shape
#tile2_poly.shape
#tile2_points.shape
#tile2_cr.shape


(9320L, 7744L, 3L)

In [25]:
"""
import gdal
from gdalconst import GA_ReadOnly
data = gdal.Open(training_pts, GA_ReadOnly)
geoTransform = data.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * data.RasterXSize
miny = maxy + geoTransform[5] * data.RasterYSize
'gdal_translate -projwin ' + ' '.join([str(x) for x in [minx, maxy, maxx, miny]]) + ' -of GTiff img_orig.tif img_out.tif', shell=True)
"""

"\nimport gdal\nfrom gdalconst import GA_ReadOnly\ndata = gdal.Open(training_pts, GA_ReadOnly)\ngeoTransform = data.GetGeoTransform()\nminx = geoTransform[0]\nmaxy = geoTransform[3]\nmaxx = minx + geoTransform[1] * data.RasterXSize\nminy = maxy + geoTransform[5] * data.RasterYSize\n'gdal_translate -projwin ' + ' '.join([str(x) for x in [minx, maxy, maxx, miny]]) + ' -of GTiff img_orig.tif img_out.tif', shell=True)\n"

In [16]:
"normalizing the input image"
tile2_top = nulltozero(tile2_top)
tile2_top=tile2_top/np.amax(tile2_top)


In [17]:
"reshaping the gts"
tile2_poly = np.reshape(tile2_poly,(tile2_poly.shape[0],tile2_poly.shape[1]))


In [18]:
"create an idx array by sampling from the tile2_points"

idx_tile2= sample_idx(tile2_points[:,:,0])

In [19]:
#idx_tile2.shape

(2987L, 2L)

In [20]:
#proportion = 0.995
"create the tr and ts idx"
#(tr_idx_tile2, ts_idx_tile2)= split_samples(idx_tile2,proportion)
# defining the ids of the training samples
tr_idx_tile2 = idx_tile2


In [30]:
psize = 33
#nbands = top_cl_1.shape[2] + 1
ignoreedge = 0

nbands = tile2_top.shape[2] + 1
trainset = np.zeros(shape=(0, psize, psize, nbands))                
#testset = np.zeros(shape=(0, psize, psize, nbands))

"accumulate samples for tile 2"
(trainset,trainidx) = accum_samples_cnn(trainset,(tile2_top,tile2_poly),
											   tr_idx_tile2,
											   psize, ignoreedge=ignoreedge)
#(testset,testidx) = accum_samples_cnn(testset,(tile2_top,tile2_poly),
#											  ts_idx_tile2,
#											  psize, ignoreedge=ignoreedge)

f0 shape (9320L, 7744L, 3L)
ncols 4
nonedgesamples[0].shape[0] 2954
nonedgefset shape (2954L, 33L, 33L, 4L)


In [31]:
nf = nbands-1											 
#reshaping the trainset and the testset
trainset = np.swapaxes(trainset, 1, 3)
trainset = np.swapaxes(trainset, 2, 3)
#dataset = trainset
trainset[:, nf, :, :] = trainset[:, nf, :, :] - 1


In [28]:
#nf = nbands-1											 
#reshaping the trainset and the testset
#trainset = np.swapaxes(trainset, 1, 3)
#trainset = np.swapaxes(trainset, 2, 3)
#dataset = trainset
#trainset[:, nf, :, :] = trainset[:, nf, :, :] - 1


In [32]:
#shape of the trainset
trainset.shape

(2987L, 4L, 33L, 33L)

In [33]:
trainset[0,0]

array([[ 0.84313726,  0.85882354,  0.86666667, ...,  0.80000001,
         0.7764706 ,  0.80784315],
       [ 0.88627452,  0.89019608,  0.88627452, ...,  0.80392158,
         0.7647059 ,  0.80000001],
       [ 0.89019608,  0.91764706,  0.84705883, ...,  0.81176472,
         0.81960785,  0.8392157 ],
       ..., 
       [ 0.73333335,  0.73725492,  0.71372551, ...,  0.94901961,
         0.98039216,  1.        ],
       [ 0.71372551,  0.74901962,  0.70588237, ...,  0.93725491,
         0.96078432,  0.98431373],
       [ 0.69411767,  0.73333335,  0.67843139, ...,  0.9254902 ,
         0.94117647,  0.95686275]])

In [35]:
#smpldir= rootfolder + "samples/samples_weakly_geobia2.hdf5"
#inddir = rootfolder + "samples/samples_indir_weakly_geobia2.hdf5"

trainproportion = 0.8
alltrainset = trainset
#alltestset = testset
nc=5
save_samplesets(alltrainset=alltrainset,
                   #alltestset=alltestset,
                   fname=smpldir,idfname = inddir,
                   nc=nc, tp=trainproportion)